In [15]:
import gzip, shutil
import os 
import data_utils as du 
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
from six.moves import cPickle as pickle
import numpy as np
import os
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt 
import platform
import data_utils as du 
from cnn_model import *
from solver import Solver
import qunatize_resnet20 as q_res
import quantize_layers as ql
import tensorflow_hub as hub 
import quantize_util as qu 
import quantize_complier as qc 


%load_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext autoreload
plt.rcParams['figure.figsize'] = (15.0, 15.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

Data_folder = 'CIFAR10_Data'
Data_fn = 'cifar-10-python'
X_train, y_train, X_val, y_val, X_test, y_test = du.get_CIFAR10_data(subtract_mean =True)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int64
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [16]:
data_set = {
    'train_data': X_train,
    'train_label': y_train,
    'val_data':X_val,
    'val_label':y_val,
}
train_options = {
    'optimizer': 'adam',
    'lr': 0.001,
    'epoch_num': 1,
    'from_logits':True,
    'save_dir': 'resnet20_model'
}
model_path = 'resnet20_model/trained_15'
full_model = hub.KerasLayer(model_path,trainable=True)

qunatizer = ql.ModelQunatize() 
model_q = q_res.Resnet20(qunatizer)
qu.copy_weight(model_q,full_model)
q_layers = model_q.get_quantizable_layers()
for layer in q_layers:
    print(layer)

solver = Solver(model_q,data_set,train_options,save_model=False)

data_shuffler = qc.DataShuffler(X_train, y_train,sample_size=1024)


def train_fn(model,solver=solver):
    solver = Solver(model,data_set,train_options,
                    save_model=False,verbose=False,
                    plot_graph=False)
    return solver.train(iter=int(5000/32))

def test_fn(model,solver=solver,shuffle=False):
    solver = Solver(model,data_set,save_model=False,
                    verbose=False,plot_graph=False)
    #X_s,y_s = data_shuffler.sample(shuffle)
    return solver.test_model(X_test[0:1024-1],y_test[0:1024-1])
    #return solver.test_model(X_test[0:1024*6-1],y_test[0:1024*6-1])
    #return solver.test_model(X_s,y_s)
    #return solver.test_model(X_test,y_test)






In [17]:
q_complier = qc.QunatizeComplier(model_q,full_model,q_layers,train_fn,test_fn,verbose=True,target_compression=0.12,max_loss=0.005)
q_complier.quantize_model()

Full percision model accuracy: 0.7077223658561707
************Coarse search begin************
Coarse seach done, compression rate:(0.25, 2146768),quantize model accuracy: 0.7028347849845886

************Fine search begin************

In compression search+++++++++++++++++++++++++++
reduce wl on layer 15 kernel,acc_loss: 0.00782012939453125, current accuracy:0.6999022364616394, compression rate: (0.24570703494741863, 2109904)
<quantize_complier.WlflList object at 0x7f9884ce6580>[[8,7,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][7,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,8,6]]
Compression rate:0.24570703494741863
Traget Compression rate:0.12
------Fine tune the model------
Epoch: 0/1
  156/1531 |==***********************|  - train_acc: 0.8321 - train_loss: 0.4679 - val_acc: NA    - val_loss: NA   
Epoch: 0/1
  156/1531 |==***********************|  - train_acc: 0.9050 - train_loss: 0.2791 - val_acc: NA    -

(<qunatize_resnet20.Resnet20 at 0x7f98207ef490>,
 <quantize_complier.WlflList at 0x7f9884ce6580>)

In [18]:

model_save_path = 'complier_tmp/3deep_compressed_resnet20_model'
wlfl_save_path = 'complier_tmp/3deep_compressed_resnet20_wlfl.txt'
model_q.model.save_weights(model_save_path)
tf.saved_model.save(model_q.model,model_save_path)
q_complier.wl_agent.get_wl_list().save(wlfl_save_path)


INFO:tensorflow:Assets written to: complier_tmp/3deep_compressed_resnet20_model/assets


In [19]:

model_q.model.load_weights(model_save_path)
q_layers = model_q.get_quantizable_layers()
wlfl_list = qc.WlflList(len(q_layers))
wlfl_list.load(wlfl_save_path)
q_complier.fl_agent.apply_wlfl_to_layers(q_layers,wlfl_list)
solver = Solver(model_q,data_set,train_options)
solver.test_model(X_test,y_test)
q_complier.wl_agent.get_compression_rate()


Loss: 1.9256057739257812,Test Accuracy: 66.50999450683594


(0.2457000476996117, 2109844)

In [21]:
solver.test_model(X_test,y_test)

Loss: 1.9256057739257812,Test Accuracy: 66.50999450683594


(<tf.Tensor: shape=(), dtype=float32, numpy=1.9256058>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6651>)